## An example workflow

In [1]:
from Poseidon.model import *
from Poseidon.dem import *
from Poseidon.meteo import *

### setup

In [2]:
#define in a dictionary the properties e.g
dic={'lon0':0., # lat/lon window
     'lon1':10.,
     'lat0':28.,
     'lat1':38.,
     'tag':'test',
     'date':datetime.datetime(2013,3,1,0),
     'resolution':0.1}

In [3]:
#create a d3d model
b = d3d()
b.set(**dic) #set it up 

create grid


In [4]:
#view model
#b.__dict__

### Define preferences 

In [5]:
Tstart = 0
Tstop = 60
step = 60

In [6]:
  #define grid file
b.mdf.inp['Filcco']=b.tag+'.grd'
  
  #define enc file
b.mdf.inp['Filgrd']=b.tag+'.enc'
  
  #define dep file
b.mdf.inp['Fildep']=b.tag+'.dep'
  
  #define obs file
b.mdf.inp['Filsta']='' #b.tag+'.obs'
  
  
  # adjust ni,nj
b.mdf.inp['MNKmax']=[b.ni+1,b.nj+1,1]  # add one like ddb
  
  # adjust iteration date
b.mdf.inp['Itdate']=datetime.datetime.strftime(b.date,'%Y-%m-%d')
  
  #set time unit
b.mdf.inp['Tunit']='M'

  #adjust iteration start
b.mdf.inp['Tstart']=[Tstart]
  
  #adjust iteration stop
b.mdf.inp['Tstop']=[Tstop]
  
  #adjust time step
b.mdf.inp['Dt']=[1.]
  
  #adjust time for output
step=60
b.mdf.inp['Flmap']=[Tstart,step,Tstop]
b.mdf.inp['Flhis']=[Tstart,1,Tstop]
b.mdf.inp['Flpp']=[0,0,0]
b.mdf.inp['Flrst']=[720]
  
  #time interval to smooth the hydrodynamic boundary conditions
b.mdf.inp['Tlfsmo']=[0.]


  # set tide only run
#if 'tide' in kwargs.keys() :
#     if (kwargs['tide']==True) & (force==False):
#        inp['Sub1'] = ' '
#        inp['Filbnd']=basename+'.bnd'
#        inp['Filana']=basename+'.bca'
#       if 'Tidfor' not in order: order.append('Tidfor')
#       inp['Tidfor']=[['M2','S2','N2','K2'], \
#                      ['K1','O1','P1','Q1'], \
#                      ['-----------']]
 
  # specify ini file
# if 'Filic' not in order: order.append('Filic')
# inp['Filic']=basename+'.ini'

  # netCDF output
if 'FlNcdf' not in b.mdf.order: b.mdf.order.append('FlNcdf')

b.mdf.inp['FlNcdf'] = 'map his'


### extract meteo forcing

In [7]:
#set meteo forcing
b.meteo = ecmwf() # initialize

In [8]:
# specify the forecast times ft1,ft2
b.meteo.ft1 = 0
b.meteo.ft2 = 2 #that is 0,1 hours

In [9]:
# parse the ecmwf grib file.
b.meteo.parse(path='/Users/brey/Downloads/20130301.00.tropical_cyclone.grib',**b.__dict__)

100%|██████████| 6/6 [00:02<00:00,  2.47it/s]


meteo done


In [10]:
# view results
#b.meteo.__dict__

### extract bathymetry

In [11]:
#set dem data
b.dem = gebco08(**b.__dict__) # initialize

In [12]:
b.dem.get(file='/Users/brey/Downloads/gebco30_DELTARES.nc',grid_x=b.grid.x,grid_y=b.grid.y)

In [13]:
#view model
#b.dem.__dict__

## output 

In [14]:
#create the model folder
path='/Users/brey/Downloads/test/'
if not os.path.exists(path):
    os.makedirs(path)

In [15]:
b.output(path=path)

In [16]:
# save model for further use
b.save(path=path)

### execute

In [16]:
#execute
b.run(run_path=path,d3d='/Users/brey/DELFT3D',arch='lnx64',ncores=4)

MPI process number 000 has host unknown and is running on processor Ronin.local

MPI process number 001 has host unknown and is running on processor Ronin.local

MPI process number 003 has host unknown and is running on processor Ronin.local

MPI process number 002 has host unknown and is running on processor Ronin.local

--------------------------------------------------------------------------------

       Deltares, FLOW2D3D Version 6.02.10.7072, Mar 11 2017, 09:06:36

       libflow2d3d.so entry Flow2D3D::Run

--------------------------------------------------------------------------------



Part I    - Initialisation Time Dep. Data module...    

            runid : test

Part II   - Creating intermediate files...             

Part III  - Initialisation of the Execution module...  

Part IV   - Reading complete MD-file...                     

Part V    - Initialisation & checking input...              

Part VI   - Initialisation & checking second part...        

Part VII  - I